In [1]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [2]:
import copy
import datetime
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload

import celeri
celeri = reload(celeri)

In [3]:
RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
command_file_name = './data/western_north_america/western_north_america_command.json'
# command_file_name = './data/global/global_command.json'
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
segment, station, block, mogi, sar = celeri.assign_block_labels(segment, station, block, mogi, sar)
assembly = celeri.create_assembly_dictionary()
assembly = celeri.merge_geodetic_data(assembly, station, sar)
partials = {} 
assembly, partials["block_motion_constraints"] = celeri.block_constraints(assembly, block, command)
assembly, partials["slip_rate_constraints"] = celeri.slip_rate_constraints(assembly, segment, block, command)

TypeError: process_segment() missing 1 required positional argument: 'meshes'

In [ ]:
print(len(segment))
print(len(segment_split))
